In [1]:
import bmdrc
import pandas as pd

## Module 1: Input Data Modules 

##### LPR Class

In [2]:
from bmdrc.LPRClass import LPRClass

lpr_example = pd.read_csv("/Users/degn400/Git_Repos/bmdrc/data/LPR_Long.csv")

LPR = LPRClass(
    df = lpr_example,
    chemical = "chemical.id",
    plate = "plate.id",
    well = "well",
    concentration = "conc",
    time = "variable",
    value = "value",
    cycle_length = 20.0,
    cycle_cooldown = 10.0, 
    starting_cycle = "light"
)

...defining cycles


In [3]:
LPR._df

,chemical.id,conc,plate.id,well,variable,value,cycle
0,2,0.0,20544,H01,0.0,0.000,light1
1,2,0.0,20544,H02,0.0,NaN,light1
2,2,0.0,20544,H03,0.0,NaN,light1
3,2,0.0,20544,H04,0.0,1.083,light1
4,2,0.0,20544,H05,0.0,1.608,light1
...,...,...,...,...,...,...,...
69115,2,100.0,20625,A08,239.0,0.000,gap_dark4
69116,2,100.0,20625,A09,239.0,0.000,gap_dark4
69117,2,100.0,20625,A10,239.0,0.000,gap_dark4
69118,2,100.0,20625,A11,239.0,8.332,gap_dark4


In [4]:
# Calculate AUC values 
aucs = LPR._df[~LPR._df["cycle"].str.contains("gap")].drop(labels = LPR._time, axis = 1)
aucs = aucs.groupby(by = [LPR._chemical, LPR._concentration, LPR._plate, LPR._well, "cycle"]).sum().reset_index()



In [5]:
store_aucs = []

for cycle_num in range(LPR._max_cycle):

    # Pull cycle information
    cycle_num = cycle_num + 1
    light_name = "light" + str(cycle_num)
    dark_name = "dark" + str(cycle_num)

    # Merge light and dark information
    to_calc_auc = pd.merge(
        aucs[aucs["cycle"] == light_name].rename(columns = {"value":"light"}).drop("cycle", axis = 1),
        aucs[aucs["cycle"] == dark_name].rename(columns = {"value":"dark"}).drop("cycle", axis = 1),
        how = "left"
    )

    store_aucs.append([to_calc_auc["dark"][x] - to_calc_auc["light"][x] for x in range(len(to_calc_auc))])

auc_values = pd.DataFrame(store_aucs).transpose()

for name in auc_values.columns:
    new_name = "AUC" + str(int(name) + 1)
    auc_values.rename(columns={name:new_name}, inplace = True)

auc_process = pd.concat([
    LPR._df[[LPR._chemical, LPR._concentration, LPR._plate, LPR._well]].drop_duplicates(),
    auc_values
], axis = 1)

auc_process

,chemical.id,conc,plate.id,well,AUC1,AUC2,AUC3,AUC4
0,2,0.0,20544,H01,119.195001,139.074000,78.840000,108.243
1,2,0.0,20544,H02,0.000000,0.000000,0.000000,0.000
2,2,0.0,20544,H03,0.000000,0.000000,0.000000,0.000
3,2,0.0,20544,H04,44.267998,-12.235000,158.610998,141.352
4,2,0.0,20544,H05,324.989997,331.197997,228.420001,51.068
...,...,...,...,...,...,...,...,...
283,2,100.0,20625,A08,0.276000,12.953000,7.803000,-9.328
284,2,100.0,20625,A09,-4.327000,23.387000,-5.651000,12.079
285,2,100.0,20625,A10,107.622001,97.236001,32.498001,125.039
286,2,100.0,20625,A11,51.565000,67.338000,83.437000,154.584


In [6]:
def to_dichotomous(the_value):

    LPR_plateGroups = auc_process[[LPR._chemical, LPR._plate, LPR._concentration, the_value]]
    LPR_zero = LPR_plateGroups[LPR_plateGroups[LPR._concentration] == 0].groupby([LPR._chemical, LPR._plate])

    # Pull quartile calculations
    rangeValues = LPR_zero.apply(lambda df: df[the_value].quantile(0.25)).reset_index().rename(columns = {0:"Q1"})
    rangeValues["Q3"] = LPR_zero.apply(lambda df: df[the_value].quantile(0.75)).reset_index()[0]

    # Add IQR and lower and upper bonds. 
    rangeValues["IQR"] = rangeValues["Q3"] - rangeValues["Q1"]
    rangeValues["Low"] = rangeValues["Q1"] - (1.5 * rangeValues["IQR"])
    rangeValues["High"] = rangeValues["Q3"] + (1.5 * rangeValues["IQR"])
    rangeValues = rangeValues[[LPR._chemical, LPR._plate, "Low", "High"]]

    LPR_plateGroups = pd.merge(LPR_plateGroups, rangeValues)
    LPR_plateGroups["result"] = (LPR_plateGroups[the_value] < 0) | (LPR_plateGroups[the_value] < LPR_plateGroups["Low"]) | (LPR_plateGroups[the_value] > LPR_plateGroups["High"])

    return(LPR_plateGroups["result"].astype(float))

In [7]:
AUC_Converted = []

for x in range(LPR._max_cycle):
    value = "AUC" + str(x + 1)
    auc_process[value] = to_dichotomous(value)

auc_process

,chemical.id,conc,plate.id,well,AUC1,AUC2,AUC3,AUC4
0,2,0.0,20544,H01,0.0,0.0,0.0,0.0
1,2,0.0,20544,H02,0.0,0.0,0.0,0.0
2,2,0.0,20544,H03,0.0,0.0,0.0,0.0
3,2,0.0,20544,H04,0.0,1.0,0.0,0.0
4,2,0.0,20544,H05,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
283,2,100.0,20625,A08,0.0,0.0,0.0,1.0
284,2,100.0,20625,A09,1.0,0.0,1.0,0.0
285,2,100.0,20625,A10,0.0,0.0,0.0,0.0
286,2,100.0,20625,A11,0.0,0.0,0.0,0.0


In [ ]:
self._df

In [25]:
import numpy as np

# Group by chemical, concentration, plate, well and time
LPR_grouped = LPR._df.groupby(by = [LPR._chemical, LPR._concentration, LPR._plate, LPR._well, LPR._time])

# Sum times 
LPR_sum = LPR_grouped.sum().rename(columns = {LPR._time:"Sum"}).reset_index()

# Remove times with NA 
LPR_sum["Remove"] = LPR_grouped.apply(lambda df: any(np.isnan(df[LPR._value]))).reset_index().rename(columns = {0:"Remove"})["Remove"]
LPR_sum = LPR_sum[LPR_sum["Remove"] == False].drop(columns = ["Remove"])

In [34]:
# Unique and arrange times 
cycle_info = pd.DataFrame(LPR._df[LPR._time].unique()).rename({0:LPR._time}, axis = 1)
cycle_info[LPR._time] = cycle_info[LPR._time].astype(float)
cycle_info = cycle_info.sort_values(by = [LPR._time])

# Build cycle names and order. First, define all the needed variables to make this happen
cycle_order = []
first_count = 0
gap_a_count = 0
gap_b_count = 0
second_count = 0
cycle_count = 1
if LPR._starting_cycle == "light":
    other_cycle = "dark"
else:
    other_cycle = "light"

# Cycle through the light, gap, dark, and then reset
for pos in range(len(cycle_info)):
    if (first_count < LPR._cycle_length):
        cycle_order.append(LPR._starting_cycle + str(cycle_count))
        first_count += 1
    elif (gap_a_count < LPR._cycle_cooldown):
        cycle_order.append("gap_" + LPR._starting_cycle + str(cycle_count))
        gap_a_count += 1
    elif (second_count < LPR._cycle_length):
        cycle_order.append(other_cycle + str(cycle_count))
        second_count += 1
    elif (gap_b_count < LPR._cycle_cooldown):
        cycle_order.append("gap_" + other_cycle + str(cycle_count))
        gap_b_count += 1
    else:
        cycle_count += 1
        cycle_order.append(LPR._starting_cycle + str(cycle_count))
        first_count = 1
        gap_a_count = 0
        second_count = 0
        gap_b_count = 0
    
# Add essential order information to cycle_info file
cycle_info["cycle"] = cycle_order

# Merge with data.frame
LPR._df = LPR._df.merge(cycle_info)



In [22]:
# Group by chemical, concentration, plate, and well. 
LPR_sum_grouped = LPR_sum.groupby(by = [LPR._chemical, LPR._concentration, LPR._plate, LPR._well])
